In [6]:
import pandas as pd
import os
import logging
import sys
import pyodbc

In [14]:
nombre_archivo=r'C:\Users\user\Desktop\PRUEBA_TECNICA\Ingeniero_datos_p_tecnica\data\raw'

In [15]:
def conf_logging (log_archivo:str):#conf_logging (log_archivo):

    logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_archivo, mode='a',encoding="utf-8"),  # Guardar log en archivo
        logging.StreamHandler(),  # También quiero imprimir log en la consola
    ],
    )
    logger = logging.getLogger()
    return logger

In [25]:
try:
    conn = pyodbc.connect(
        "DRIVER={PostgreSQL ODBC Driver(UNICODE)};"
        "SERVER=localhost;"
        "DATABASE=pipeline_db;" 
        "PORT=5433;"
        "UID=postgres;"
        "PWD=postgres"
    )
    logging.info("Conexión a la base de datos sumi exitosa.")
except pyodbc.Error as e:
    logging.error(f"Error al conectar con la base de datos sumi: {e}")
    raise

2025-07-28 00:11:46,837 - INFO - Conexión a la base de datos sumi exitosa.


In [16]:
logger = conf_logging("notbook_log")

In [22]:
nombre_archivo_2=os.path.join(nombre_archivo, 'data_jobs.csv')
try:
    df_data=pd.read_csv(f'{nombre_archivo_2}', sep=',',low_memory=False)
except Exception as e:
    logger.error(f"Error al cargar el archivo: {e}")
    raise

In [27]:
def mapeo_tipo_dato(tipos, muestra=None):
        if pd.api.types.is_integer_dtype(tipos):
            return 'INTEGER'
        elif pd.api.types.is_float_dtype(tipos):
            return 'FLOAT'
        elif pd.api.types.is_bool_dtype(tipos):
            return "BOOLEAN"
        elif pd.api.types.is_datetime64_any_dtype(tipos):
            return "TIMESTAMP"
        elif pd.api.types.is_object_dtype(tipos) and isinstance(muestra, list):
            return "TEXT"
        elif pd.api.types.is_object_dtype(tipos) and isinstance(muestra, dict):
            return "JSONB"
        elif pd.api.types.is_string_dtype(tipos):
            return "VARCHAR(255)"
        else:
            return "TEXT"

def crear_tabla_dinamica(conn,df, tabla):
    cursor = conn.cursor()
    columnas = []
    
    for col in df.columns:
        tipo_pg = mapeo_tipo_dato(df[col].dtype, df[col].dropna().iloc[0] if not df[col].dropna().empty else None)
        col_limpio = col.lower().strip().replace(" ", "_")
        columnas.append(f'"{col_limpio}" {tipo_pg}')
    
    columnas_sql = ", ".join(columnas)
    sql = f"""
        CREATE TABLE IF NOT EXISTS {tabla} (
            {columnas_sql}
        );
    """
    return sql
    #cursor.execute(sql)
    #self.conn.commit()

In [28]:
a=crear_tabla_dinamica(conn, df_data, 'data_jobs')

In [29]:
a

'\n        CREATE TABLE IF NOT EXISTS data_jobs (\n            "job_title_short" VARCHAR(255), "job_title" VARCHAR(255), "job_location" VARCHAR(255), "job_via" VARCHAR(255), "job_schedule_type" VARCHAR(255), "job_work_from_home" BOOLEAN, "search_location" VARCHAR(255), "job_posted_date" VARCHAR(255), "job_no_degree_mention" BOOLEAN, "job_health_insurance" BOOLEAN, "job_country" VARCHAR(255), "salary_rate" VARCHAR(255), "salary_year_avg" FLOAT, "salary_hour_avg" FLOAT, "company_name" VARCHAR(255), "job_skills" VARCHAR(255), "job_type_skills" VARCHAR(255)\n        );\n    '